In [5]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
def init_browser():
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    return Browser("chrome", **executable_path, headless=False)

In [75]:
def scrape():
    browser = init_browser()
    nasa_data = {}
    url1 = 'https://mars.nasa.gov/news/'
    url2 = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
    
    browser.visit(url1)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    #collect the latest news title and paragraph text (latest result will be first in list; index 0 of the scaping results)
    nasa_data["title"] = soup.find_all("div", class_="list_text")[0].find("a").text
    nasa_data["paragraph"] = soup.find_all("div", class_="list_text")[0].find("div", class_="article_teaser_body").text
    
    browser.visit(url2)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    #scrape image URL from website
    relative_image_path = soup.find_all("img", class_="headerimage fade-in")[0]["src"]
    #concatenate relative image path to website URL
    nasa_data["featured_image_url"] = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/" + relative_image_path
    
    browser.quit()
    
    return nasa_data

In [76]:
scrape()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Garrett_Personal\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


{'title': "NASA's Perseverance Pays Off Back Home",
 'paragraph': 'Even as the Perseverance rover approaches Mars, technology on board is paying off on Earth.',
 'featured_image_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'}